In [1]:
import torch
from jupyter_lsp.specs import json
from sklearn.preprocessing import LabelEncoder

print("cuda" if torch.cuda.is_available() else "cpu")

cuda


In [1]:
#import library
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import re
import emoji
from underthesea import text_normalize



F:\ds_competition\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
data_predict_path = {
    'json': r"F:\ds_competition\Data_test_public\vimmsd-public-test.json",
    'images': r"F:\ds_competition\Data_test_public\public-test-images\dev-images"
}

data_path = {
    'json': r'F:\ds_competition\Data_train\vimmsd-train.json',
    'images': r'F:\ds_competition\Data_train\training-images\train-images'
}

# Them text_sarcasm

In [7]:
with open(data_path['json'], 'r', encoding='utf-8') as f:
    data = json.load(f)
    

In [8]:
# Tìm và giữ lại các mục không có label là 'text-sarcasm'
filtered_data = {key: value for key, value in data.items() if value.get('label') != 'text-sarcasm'}

# Đánh số lại các key từ 0
updated_data = {str(index): value for index, (key, value) in enumerate(filtered_data.items())}

In [9]:
len(updated_data)

10728

In [10]:
with open(r'F:\ds_competition\results\augmented_data_text_sc.json', 'r', encoding='utf-8') as f:
    data_aug_text_sarcasm = json.load(f)

In [11]:
# Tìm key lớn nhất của data1 và bắt đầu từ đó
max_key = max(map(int, updated_data.keys())) if updated_data else -1
max_key

10727

In [12]:
# Tăng key của data2 lên và nối vào data1
for i, (key, value) in enumerate(data_aug_text_sarcasm.items(), start=max_key + 1):
    updated_data[str(i)] = value

In [13]:
len(updated_data)

11094

In [14]:
# Ghi dữ liệu hợp nhất vào file mới
with open('data_augment_text_sarcasm.json', 'w', encoding='utf-8') as f_out:
    json.dump(updated_data, f_out, indent=4, ensure_ascii=False)

# ádfasdfasdf

In [2]:
def load_data(path):
    with open(path, 'r', encoding='utf-8') as f:
        f = json.load(f)
    data = pd.DataFrame.from_dict(f, orient='index')
    return data


In [3]:
df = load_data(r'F:\ds_competition\Data_private\vimmsd-private-test.json')
df.head()

,image,caption,label
0,066d6021fdfeaf39f1dec523879e8fe4d35e877abcea44...,Song Joong Ki &amp; Song Hye Kyo đều tham dự B...,None
1,555f4787d4df49e7be743b3d5b77c90755f0d6c351f36b...,Song Joong Ki &amp; Song Hye Kyo đều tham dự B...,None
2,7b7cdea2cde1f3f93371259b587a03f2e8c0af682b4d51...,Song Joong Ki &amp; Song Hye Kyo đều tham dự B...,None
3,80167e59d729cf3aaba5d2d3da40db6995cb8a6a8c4a88...,Song Joong Ki &amp; Song Hye Kyo đều tham dự B...,None
4,59db087307031d60755af3a5c01a44ba55a04bfab21027...,Ngang trái thậc 🤣,None


In [15]:
import pandas as pd

# Giả sử df là DataFrame ban đầu với các cột 'caption' và 'label'
# Lọc các giá trị trùng lặp trong cột 'caption'
duplicates = df[df.duplicated(subset=['caption'], keep=False)]

# Đếm số lần xuất hiện cho từng loại label trên các giá trị 'caption' bị trùng
result = (
    duplicates.groupby(['caption', 'label'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
    .rename_axis(None, axis=1)
)

# Đổi tên các cột để dễ hiểu hơn
result.columns = ['caption', 'not-sarcasm', 'text-sarcasm', 'image-sarcasm', 'multi-sarcasm']


In [17]:
# Xuất DataFrame 'result' ra file CSV
result.to_csv('caption_duplicates.csv', index=False)

In [58]:
result

,caption,not-sarcasm,text-sarcasm,image-sarcasm,multi-sarcasm
0,"""BƠI ĐÊM"" CÓ NGÀY GẶP ...?\nGia đình Waller gồ...",0,0,4,0
1,"""Lấy Mẻ bôi kín những chỗ bám bẩn xong để qua ...",0,0,2,0
2,"""MẠNG XÃ HỘI ZING ME CHÍNH THỨC RA MẮT GIỚI TR...",0,0,5,0
3,"""Những tổn thương ấy vẫn luôn ở đây, chẳng qua...",0,0,4,0
4,"""Tý về ông già chửi tau chớt luôn á"" - Con giờ...",0,0,4,0
...,...,...,...,...,...
692,🥕🥕RA MẮT ÁO THUN STREET STYLE THỎ BẢY MÀU SS1🥕...,0,0,5,0
693,🥰,2,0,1,0
694,🥰🥰😍😍🥰🥰😍😌😍🥰🥰🥲🙂😚,0,0,5,0
695,🥲,0,2,0,0


In [47]:
duplicate_captions = result['caption'].tolist()

# handle duplicate with ocr

In [ ]:
# def handle_duplicate_with_ocr(input_path, output_path, duplicate_captions = duplicate_captions):
#     with open(input_path, 'r', encoding='utf-8') as f:
#         data = json.load(f)
#         
#         for key, value in data.items():
#             if value['caption'] in duplicate_captions:
#                 value['caption'] = value['text_image'] + ' ' + value['caption']
#         
#     # Ghi dữ liệu ra file JSON mới
#     with open(output_path, 'w', encoding='utf-8') as file:
#         json.dump(data, file, ensure_ascii=False, indent=4)
#         
# input_train_path = r'F:\ds_competition\processed_train_demoji_remove_hashtag_url_have_orc.json'
# output_train_path = 'updated_data_train_handle_duplicates.json'
# handle_duplicate_with_ocr(input_train_path, output_train_path)
# 
# input_pred_path = r''

In [ ]:
df.info()

In [ ]:
df['label'].unique()

In [8]:
encoded_label = {
    'not-sarcasm': 0,
    'image-sarcasm': 1,
    'text-sarcasm': 2,
    'multi-sarcasm': 3,
}
labels = ['multi-sarcasm', 'not-sarcasm', 'image-sarcasm', 'text-sarcasm']

df['encoded_label'] = df['label'].apply(lambda x: encoded_label[x])


In [ ]:
df['label'].value_counts()

In [10]:
# from transformers import AutoTokenizer
# 
# # Tải tokenizer cho PhoBERT
# tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
# 
# # Tính số lượng token cho mỗi caption
# df['num_token_bert'] = df['caption'].apply(lambda x: len(tokenizer(x)['input_ids']))


In [11]:
df.describe()

,encoded_label
count,10805.00000
mean,1.22795
std,1.44226
min,0.00000
25%,0.00000
50%,0.00000
75%,3.00000
max,3.00000


In [12]:
# df['num_token_bert'].value_counts()

# phân tích emoji

In [13]:
df.head()

,image,caption,label,encoded_label
0,8ae451edcd8ebf697f8763ece249115813149c55733bf8...,Cô ấy trên mạng vs cô ấy ngoài đời =))),multi-sarcasm,3
1,35370ffd6c791d6f8c4ab3dd4363ed468fab41e4824ee9...,Người tâm linh giao tiếp với người thực tế :))),not-sarcasm,0
2,316fdd1477725b9fb1a55015ac06b68b92b50bd4303e08...,Hình như Trăng hôm nay đẹp quá mọi người ạ! 😃 ...,multi-sarcasm,3
3,8a0f34e0e30e4e5cfb306933c1d25fa801a5da78646b59...,MỌI NGƯỜI NGHĨ SAO VỀ PHÁT BIỂU CỦA SHARK VIỆT...,not-sarcasm,0
4,e517a5e95d1065886a7c815e82fe254381d4f9f4b244d4...,2 tay hai nàng chứ việc gì phải lệ hai hàng,multi-sarcasm,3


In [8]:
import emoji

# Tạo DataFrame để lưu trữ kết quả
df_emoji = pd.DataFrame(columns=['emoji'])


# Hàm để tìm emoji trong một chuỗi
def extract_emojis(text):
    return [char for char in text if emoji.is_emoji(char)]


labels = ['multi-sarcasm', 'not-sarcasm', 'text-sarcasm', 'image-sarcasm']
# Đếm số lần xuất hiện của mỗi emoji trong từng lớp
for label in labels:
    # Lọc DataFrame theo label
    # filtered_df = df[df['label'] == label]

    # Tìm tất cả các emoji trong cột và tách ra thành từng hàng
    all_emojis = df['caption'].apply(extract_emojis).explode()

    # Đếm số lần xuất hiện của từng emoji
    emoji_counts = all_emojis.value_counts().reset_index()
    emoji_counts.columns = ['emoji', label]
    df_emoji = df_emoji.merge(emoji_counts, on='emoji', how='outer')

df_emoji = df_emoji.fillna(0)

In [9]:
df_emoji

,emoji,multi-sarcasm,not-sarcasm,text-sarcasm,image-sarcasm
0,‼,7,7,7,7
1,™,1,1,1,1
2,☠,5,5,5,5
3,☺,1,1,1,1
4,♂,2,2,2,2
...,...,...,...,...,...
98,🫡,1,1,1,1
99,🫰,4,4,4,4
100,🫳,1,1,1,1
101,🫵,4,4,4,4


In [16]:
df_emoji['multi_rate'] = df_emoji['multi-sarcasm'].apply(lambda x : x * 4224/10805)
df_emoji['not_rate'] = df_emoji['not-sarcasm'].apply(lambda x : x * 6062/10805)
df_emoji['image_rate'] = df_emoji['image-sarcasm'].apply(lambda x : x * 442/10805)
df_emoji['text_rate'] = df_emoji['text-sarcasm'].apply(lambda x : x * 77/10805)


In [17]:
df_emoji

,emoji,multi-sarcasm,not-sarcasm,text-sarcasm,image-sarcasm,multi_rate,not_rate,image_rate,text_rate
0,©,3.0,7.0,0.0,0.0,1.172790,3.927256,0.000000,0.0
1,‼,3.0,46.0,0.0,1.0,1.172790,25.807682,0.040907,0.0
2,⁉,1.0,5.0,0.0,0.0,0.390930,2.805183,0.000000,0.0
3,↕,1.0,0.0,0.0,0.0,0.390930,0.000000,0.000000,0.0
4,⏰,2.0,23.0,0.0,0.0,0.781860,12.903841,0.000000,0.0
...,...,...,...,...,...,...,...,...,...
511,🫦,1.0,0.0,0.0,0.0,0.390930,0.000000,0.000000,0.0
512,🫨,0.0,5.0,0.0,0.0,0.000000,2.805183,0.000000,0.0
513,🫰,5.0,33.0,0.0,0.0,1.954651,18.514206,0.000000,0.0
514,🫵,1.0,16.0,0.0,1.0,0.390930,8.976585,0.040907,0.0


sự phân phối của emoji không đồng đều có thể dẫn đến mô hình sẽ dựa vào emoji để dự đoán mà không xét tới dấu hiệu đa phương thức </br>
**Đề xuất:** Loại bỏ emoji

# Phân tích hashtag

In [18]:
df_hashtags = pd.DataFrame(columns=['hashtag'])


def extract_hashtags(text):
    hashtag = re.findall(r'#\w+', text)
    return hashtag


for label in labels:
    # Lọc DataFrame theo label
    filtered_df = df[df['label'] == label]

    # Tìm tất cả các emoji trong cột và tách ra thành từng hàng
    all_hashtags = filtered_df['caption'].apply(extract_hashtags).explode()

    # Đếm số lần xuất hiện của từng emoji
    hashtag_counts = all_hashtags.value_counts().reset_index()
    hashtag_counts.columns = ['hashtag', label]
    df_hashtags = df_hashtags.merge(hashtag_counts, on='hashtag', how='outer')

df_hashtags = df_hashtags.fillna(0)

In [19]:
df_hashtags['multi_rate'] = df_hashtags['multi-sarcasm'].apply(lambda x : x * 4224/10805)
df_hashtags['not_rate'] = df_hashtags['not-sarcasm'].apply(lambda x : x * 6062/10805)
df_hashtags['image_rate'] = df_hashtags['image-sarcasm'].apply(lambda x : x * 442/10805)
df_hashtags['text_rate'] = df_hashtags['text-sarcasm'].apply(lambda x : x * 77/10805)


In [20]:
df_hashtags

,hashtag,multi-sarcasm,not-sarcasm,text-sarcasm,image-sarcasm,multi_rate,not_rate,image_rate,text_rate
0,#0,1.0,0.0,0.0,0.0,0.39093,0.000000,0.000000,0.0
1,#098,0.0,1.0,0.0,0.0,0.00000,0.561037,0.000000,0.0
2,#1,1.0,16.0,0.0,0.0,0.39093,8.976585,0.000000,0.0
3,#10,0.0,0.0,0.0,1.0,0.00000,0.000000,0.040907,0.0
4,#100,0.0,1.0,0.0,0.0,0.00000,0.561037,0.000000,0.0
...,...,...,...,...,...,...,...,...,...
1557,#đọcvịquaXEseries,0.0,0.0,0.0,1.0,0.00000,0.000000,0.040907,0.0
1558,#ỚiGojeknha,0.0,1.0,0.0,0.0,0.00000,0.561037,0.000000,0.0
1559,#Ởđâucần,0.0,1.0,0.0,0.0,0.00000,0.561037,0.000000,0.0
1560,#ỞđócóMB,0.0,1.0,0.0,0.0,0.00000,0.561037,0.000000,0.0


In [26]:
df_hashtags.describe()

,multi-sarcasm,not-sarcasm,text-sarcasm,image-sarcasm,multi_rate,not_rate,image_rate,text_rate
count,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000
mean,1.000640,1.796415,0.028169,0.206786,0.391180,1.007854,0.008459,0.000201
std,9.389398,5.772722,0.278139,1.747004,3.670599,3.238708,0.071465,0.001982
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.561037,0.000000,0.000000
75%,1.000000,1.000000,0.000000,0.000000,0.390930,0.561037,0.000000,0.000000
max,296.000000,149.000000,5.000000,41.000000,115.715317,83.594447,1.677186,0.035632


In [27]:
sum(x for x in df_hashtags['multi-sarcasm'])

1563.0

In [28]:
sum(x for x in df_hashtags['not-sarcasm'])

2806.0

In [29]:
sum(x for x in df_hashtags['text-sarcasm'])

44.0

In [30]:
sum(x for x in df_hashtags['image-sarcasm'])

323.0

# Remove Emoji and Hashtag


In [15]:
import re
import emoji
from underthesea import text_normalize

def remove_hashtags_and_emojis(input_file, output_file):
    """
    Hàm để loại bỏ hashtag, emoji, URL và chuẩn hóa văn bản trong caption từ file JSON.

    Parameters:
    - input_file: đường dẫn đến file JSON đầu vào.
    - output_file: đường dẫn đến file JSON đầu ra.
    """
    
    # Đọc file JSON
    with open(input_file, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Hàm để loại bỏ hashtag, emoji, URL và chuẩn hóa văn bản
    def clean_caption(caption):
        # Chuyển đổi caption sang chữ thường
        caption = caption.lower()
        
        # Thay thế biểu tượng cảm xúc bằng emoji, không thay thế nếu là thời gian
        # Đầu tiên, loại bỏ các thời gian khỏi caption để không bị thay thế nhầm
        # caption_without_time = re.sub(r'\b\d{1,2}[:]\d{2}\b', '', caption)

        # # Thay thế biểu tượng cảm xúc bằng emoji
        emoticons = [
            # (r"={1,}\)+", "😆"),
            # (r":{1,}\)+", "😂"), 
            # (r":{1,}\(+", "😞"), 
            # (r";{1,}\)+", "😉"),
            # (r":’\(+", "😢"),
            # (r":3", "coy_smile"),
            # (r"\(y\)", "👍"),
            # (r"._.", '😕'),
            # (r":&lt;" , "😔"),
            # (r":&gt;" , "😏"),
            (r"&amp;", '&')
        ]

        # Thay thế tất cả biểu tượng cảm xúc trong danh sách
        for pattern, emoji_symbol in emoticons:
            caption = re.sub(pattern, emoji_symbol, caption)

        # Loại bỏ hashtag
        caption = re.sub(r'#\S+', '', caption)  # Loại bỏ hashtag và các từ liền theo sau

        # Loại bỏ URL
        # caption = re.sub(r'http\S+|www\S+', '', caption)  # Loại bỏ URL

        # # Chuyển đổi emoji thành dạng văn bản
        caption = emoji.demojize(caption)
        
        # Loại bỏ emoji khỏi chuỗi văn bản
        # caption = emoji.replace_emoji(caption, replace='')
        
        # Loại bỏ \n và các ký tự không cần thiết
        caption = caption.replace('\n', '')
        # caption = re.sub(r'[.,\\,:,=,]', '', caption)
        
        return caption.strip()  

    # Cập nhật caption trong dữ liệu
    for key, entry in data.items():
        entry['caption'] = clean_caption(entry['caption'])

    # Ghi dữ liệu đã cập nhật vào file JSON mới
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)


In [16]:
remove_hashtags_and_emojis(r'F:\ds_competition\data_augment_text_sarcasm.json', r'F:\ds_competition\data_train_augment_text_sarcasm\processed_data_augment_text_sarcasm.json')

In [5]:
remove_hashtags_and_emojis(data_path['json'], 'processed_train_demove_remove_hashtag_url_v8.json')
remove_hashtags_and_emojis(data_predict_path['json'], 'processed_pred_demove_remove_hashtag_url_v8.json')
remove_hashtags_and_emojis(r'F:\ds_competition\Data_private\vimmsd-private-test.json', 'processed_private_demoji_remove_hashtag_url_v8.json')

In [54]:
# Ví dụ sử dụng hàm
output_train_json_file = 'processed_train_demoji_remove_hashtag_url_have_orc.json'  # Đường dẫn đến file JSON đầu ra
input_train_json_file = r'F:\ds_competition\Data_train\vimmsd-train-with_ocr.json'
remove_hashtags_and_emojis(input_train_json_file, output_train_json_file)

In [55]:
output_pred_json_file = 'processed_pred_demoji_remove_hashtag_url_have_ocr.json'
input_pred_json_file = r'F:\ds_competition\Data_test_public\vimmsd-public-test-with-ocr.json'
remove_hashtags_and_emojis(input_pred_json_file, output_pred_json_file)

# xử lý trùng lặp bằng ocr

In [56]:
import json

def handle_duplicate_with_ocr(input_path, output_path):
    # Đọc dữ liệu từ file JSON
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Tìm các caption trùng lặp
    captions = [value['caption'] for value in data.values()]
    duplicate_captions = [caption for caption in captions if captions.count(caption) > 1]

    # Danh sách lưu trữ các từ bị duplicate
    duplicate_words = set()

    # Duyệt qua dữ liệu và cập nhật caption nếu có trùng lặp
    for key, value in data.items():
        if value['caption'] in duplicate_captions:
            # Nối 'text_image' và 'caption' nếu trùng lặp
            value['caption'] = value['text_image'] + ' ' + value['caption']

    # Ghi dữ liệu ra file JSON mới
    with open(output_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)


In [57]:
handle_duplicate_with_ocr(output_train_json_file, 'updated_data_train_handle_duplicates.json')
handle_duplicate_with_ocr(output_pred_json_file, 'updated_data_pred_handle_duplicates.json')

In [59]:
remove_hashtags_and_emojis('updated_data_train_handle_duplicates.json', 'updated_data_train_handle_duplicates.json')
remove_hashtags_and_emojis('updated_data_pred_handle_duplicates.json', 'updated_data_pred_handle_duplicates.json')

In [ ]:
# df_processed = load_data(r'F:\ds_competition\processed_train_no_emoji_hashtag_and_have_augment.json')

In [32]:
processed_df_emoji = pd.DataFrame(columns=['emoji'])

labels = ['multi-sarcasm', 'not-sarcasm', 'text-sarcasm', 'image-sarcasm']
# Đếm số lần xuất hiện của mỗi emoji trong từng lớp
for label in labels:
    # Lọc DataFrame theo label
    filtered_df = df_processed[df_processed['label'] == label]

    # Tìm tất cả các emoji trong cột và tách ra thành từng hàng
    all_emojis = filtered_df['caption'].apply(extract_emojis).explode()

    # Đếm số lần xuất hiện của từng emoji
    emoji_counts = all_emojis.value_counts().reset_index()
    emoji_counts.columns = ['emoji', label]
    processed_df_emoji = processed_df_emoji.merge(emoji_counts, on='emoji', how='outer')

processed_df_emoji = processed_df_emoji.fillna(0)

In [33]:
processed_df_emoji

,emoji,multi-sarcasm,not-sarcasm,text-sarcasm,image-sarcasm
0,©,3.0,7.0,3.0,7.0
1,‼,3.0,46.0,3.0,35.0
2,⁉,1.0,5.0,1.0,3.0
3,↕,1.0,0.0,1.0,0.0
4,⏰,2.0,23.0,2.0,14.0
...,...,...,...,...,...
511,🫦,1.0,0.0,1.0,0.0
512,🫨,0.0,5.0,0.0,0.0
513,🫰,5.0,33.0,5.0,28.0
514,🫵,1.0,16.0,1.0,1.0


In [34]:
df_processed

,image,caption,label,text_image
0,8ae451edcd8ebf697f8763ece249115813149c55733bf8...,cô ấy trên mạng vs cô ấy ngoài đời =))),multi-sarcasm,
1,35370ffd6c791d6f8c4ab3dd4363ed468fab41e4824ee9...,người tâm linh giao tiếp với người thực tế :))),not-sarcasm,Chốn Lông Lày Sukinchana bàn thờ đột nhiên bị ...
2,316fdd1477725b9fb1a55015ac06b68b92b50bd4303e08...,hình như trăng hôm nay đẹp quá mọi người ạ! 😃 ...,multi-sarcasm,
3,8a0f34e0e30e4e5cfb306933c1d25fa801a5da78646b59...,mọi người nghĩ sao về phát biểu của shark việt...,not-sarcasm,SHARK VIỆT: ?NHIỀU NGƯỜI Ở VIỆT NAM ĐI LÀM CHỈ...
4,e517a5e95d1065886a7c815e82fe254381d4f9f4b244d4...,2 tay hai nàng chứ việc gì phải lệ hai hàng,multi-sarcasm,
...,...,...,...,...
19248,7d98022c0611752eb6bc98107000496abcd039f22f1ab9...,phát hiện vua biển atlantis arthur curry cùng ...,image-sarcasm,
19249,92f3f175dcc8185b62c91d0008df773c69f674bb507d42...,phát hiện vua biển atlantis arthur curry cùng ...,image-sarcasm,
19250,e879f0d441c5cf2837ccac8ffb463ecfaa3544745781b2...,phát hiện vua biển atlantis arthur curry cùng ...,image-sarcasm,Chảo bê con
19251,b04c39fe7ac657baaa509d674fac8310aa0b31b3f7d885...,theo trend,image-sarcasm,Bạch Trung 1 giờ - Q Minh bị đuối khỏi quán vì...


In [3]:
import json
from collections import Counter

# Giả sử file JSON của bạn có tên là 'results.json'
with open(r'F:\ds_competition\results.json', 'r') as file:
    data = json.load(file)

# Lấy danh sách các nhãn từ dictionary
labels = list(data["results"].values())

# Đếm số lượng từng loại nhãn
label_counts = Counter(labels)

# In kết quả
print(label_counts)

FileNotFoundError: [Errno 2] No such file or directory: 'F:\\ds_competition\\results.json'

In [4]:
import json

# Hàm đọc dữ liệu từ tệp JSON
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Hàm so sánh hai tệp JSON và tìm sự khác nhau
def compare_json_files(file1_path, file2_path):
    json1 = load_json(file1_path)
    json2 = load_json(file2_path)

    # Lấy kết quả từ khóa "results"
    results1 = json1.get("results", {})
    results2 = json2.get("results", {})

    # Tìm các phần tử khác nhau
    differences = {}
    for key in set(results1.keys()).union(results2.keys()):
        if results1.get(key) != results2.get(key):
            differences[key] = {
                "file1": results1.get(key),
                "file2": results2.get(key)
            }

    # In kết quả
    if differences:
        print(f"Các phần tử khác nhau giữa hai tệp: {len(differences)}")
        for key, value in differences.items():
            # if value['file2'] == 'image-sarcasm':
                print(f"Key {key}: file1 = {value['file1']}, file2 = {value['file2']}")
    else:
        print("Hai tệp giống nhau hoàn toàn!")

In [5]:
import json
from collections import Counter


# Sử dụng hàm để so sánh hai file JSON
compare_json_files(r'F:\ds_competition\TA_result_01 (1)\results.json', r'F:\ds_competition\TA_result_02 (1)\results.json')

Các phần tử khác nhau giữa hai tệp: 141
Key 1016: file1 = not-sarcasm, file2 = multi-sarcasm
Key 420: file1 = multi-sarcasm, file2 = not-sarcasm
Key 37: file1 = multi-sarcasm, file2 = not-sarcasm
Key 141: file1 = not-sarcasm, file2 = multi-sarcasm
Key 929: file1 = not-sarcasm, file2 = multi-sarcasm
Key 165: file1 = not-sarcasm, file2 = multi-sarcasm
Key 419: file1 = multi-sarcasm, file2 = not-sarcasm
Key 856: file1 = not-sarcasm, file2 = image-sarcasm
Key 1192: file1 = multi-sarcasm, file2 = not-sarcasm
Key 1179: file1 = not-sarcasm, file2 = multi-sarcasm
Key 591: file1 = not-sarcasm, file2 = multi-sarcasm
Key 223: file1 = not-sarcasm, file2 = multi-sarcasm
Key 893: file1 = multi-sarcasm, file2 = not-sarcasm
Key 952: file1 = not-sarcasm, file2 = multi-sarcasm
Key 214: file1 = multi-sarcasm, file2 = not-sarcasm
Key 1194: file1 = multi-sarcasm, file2 = not-sarcasm
Key 982: file1 = multi-sarcasm, file2 = not-sarcasm
Key 705: file1 = not-sarcasm, file2 = multi-sarcasm
Key 1095: file1 = not